In [17]:
#1/30/24 Calculate RMSD for poses- last entry of xt
#Confirm matches last entry in csv file
#Also look into csv file statistics
#Thank you Hannes for advice!
#1/31/24 from RMSD_Calc_Poses_240130_1820, streamline to make all plots in 1 doc
#2/1/24 from Streamline_Analysis_240131_1535, update to use PDB bonds only as the bonds analyzed, to prepare for consensus distance analysis
#2/4/24 from Streamline_Analysis_240201_2050, to add in conformer analysis
import glob
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import AllChem, GetPeriodicTable, RemoveHs
from rdkit.Chem import Descriptors
from rdkit.Chem import rdDistGeom
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
from scipy import stats

In [18]:
#Copied from https://github.com/gcorso/DiffDock/blob/main/datasets/process_mols.py
def read_molecule(molecule_file, sanitize=False, calc_charges=False, remove_hs=False):
    if molecule_file.endswith('.mol2'):
        mol = Chem.MolFromMol2File(molecule_file, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(molecule_file, sanitize=False, removeHs=False)
        mol = supplier[0]
    elif molecule_file.endswith('.pdbqt'):
        with open(molecule_file) as file:
            pdbqt_data = file.readlines()
        pdb_block = ''
        for line in pdbqt_data:
            pdb_block += '{}\n'.format(line[:66])
        mol = Chem.MolFromPDBBlock(pdb_block, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(molecule_file, sanitize=False, removeHs=False)
    else:
        raise ValueError('Expect the format of the molecule_file to be '
                         'one of .mol2, .sdf, .pdbqt and .pdb, got {}'.format(molecule_file))

    try:
        if sanitize or calc_charges:
            Chem.SanitizeMol(mol)

        if calc_charges:
            # Compute Gasteiger charges on the molecule.
            try:
                AllChem.ComputeGasteigerCharges(mol)
            except:
                warnings.warn('Unable to compute charges for the molecule.')

        if remove_hs:
            mol = Chem.RemoveHs(mol, sanitize=sanitize)
    except Exception as e:
        print(e)
        print("RDKit was unable to read the molecule.")
        return None

    return mol

In [23]:
multiple_molec_list = []
multiple_molec_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Analysis/Output/240205"

#This has processed PDBbind data
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed/"

#Check each complex
for ip, pdb_dir_name in enumerate(glob.glob(f"{pdb_dir}/*")):
    #print(pdb_dir_name)
    pdb_name = pdb_dir_name.split("/")[-1]
    
    #Progress track
    #if ip % 10 == 0:
    #print(f"On {pdb_name} index {ip}")
    #print(f"+++++++\n{pdb_name}\n+++++++")
    #Copied below from https://github.com/gcorso/DiffDock/blob/main/datasets/pdbbind.py
    for file in os.listdir(os.path.join(pdb_dir, pdb_name)):
        if file.endswith(".sdf") and 'rdkit' not in file:
            
            #list_pdb_names.append(pdb_name)
            
            lig = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=False, sanitize=True)
            #Pull w/o H for rmsd comparison w/HF output
            ##lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=True, sanitize=True)
            if lig is None and os.path.exists(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2")):  # read mol2 file if sdf file cannot be sanitized
                print('Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.')
                lig = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=False, sanitize=True)
                #lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=True, sanitize=True)
            
            #ref https://www.rdkit.org/docs/GettingStartedInPython.html#molecular-fragments
            #ref https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html
            l_smiles = Chem.MolToSmiles(lig)
            if "." in l_smiles:
                print("SMILES")
                print(l_smiles)
                print(f"pdb id {pdb_name} index {ip}")
                multiple_molec_list.append(pdb_name)
                
print("multiple molecs are")
print(multiple_molec_list)

[01:43:18] Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 31 32 33 34
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 60 61 62 63 64
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 55 56 57 58 59
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 16 17 18 23 24
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5 6 8 9 10
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 33 34 35 37 38 39 40 51 52
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 13 28 29 30 31
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19 21 22 23 24
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 1


Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 31 32 33 34
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 60 61 62 63 64
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 55 56 57 58 59
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 16 17 18 23 24
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5 6 8 9 10
RDKit was unable to read the molec

[01:43:18] Can't kekulize mol.  Unkekulized atoms: 79 80 81 82 83
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 63 64 65 66 67 68 69 70 71
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 67 68 69 70 71
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 12 13 14 15
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 10 11 16 17 18
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 13 20 21 22 26 27
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 5 6 24 25 26 27 28 29 40
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 0 27 28 29 30 31 32 33 34
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11 18 19 20 21
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 30 31 32 33 34
[01:43:18] Explicit valence for atom # 38 Mg, 4, is greater than permitted
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 6
[01:43:18] Ca

Can't kekulize mol.  Unkekulized atoms: 79 80 81 82 83
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 63 64 65 66 67 68 69 70 71
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 67 68 69 70 71
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 12 13 14 15
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 10 11 16 17 18
RDKit was unable to r

[01:43:18] Can't kekulize mol.  Unkekulized atoms: 38 39 40 41 42 43 44 45 46
[01:43:18] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8 10 11 12 13
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 16 17 18 19 20 23 24
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 18 19 20 21 22 23 24 25 26
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14
[01:43:19] Explicit valence for atom # 26 O, 3, is greater than permitted
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 0 2 3 4 5 14 15 16 17
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 10
[01:43:19] Explicit valence for atom # 15 O, 3, is greater than permitted
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 6 7 20 21 22 23 24 34 35
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
[01:43:19] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24
[01:43:19] Can't kekulize mol.  Unkekuliz

...

[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 9 10 11
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 3 5 9 16
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 25 26 27 28 29 30 31 32 33
[01:43:47] Explicit valence for atom # 7 O, 3, is greater than permitted
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 4 7 8 9 10
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 1 8 15 17 19 20 21 22 23
[01:43:47] Explicit valence for atom # 9 B, 4, is greater than permitted
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 14 15 16 17
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 4 12 13 19 20 21 22 23
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 11 12
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 85 86 87 88 89
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 28 29 30 31 32 33 34 35 36
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 9 10
[01

Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 9 10 11
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 3 5 9 16
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 25 26 27 28 29 30 31 32 33
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Explicit valence for atom # 7 O, 3, is greater than permitted
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 4 7 8 9 10
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 1 8 15 17 19 20 21 22 23
RDKit was unable 

[01:43:47] Explicit valence for atom # 1 B, 4, is greater than permitted
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 22 23 24 25 26 27 28 29 30
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 18 19
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 1 8 9 15 24
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 7 11 12 13
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 5 6 19 20 21
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 75 76 77 78 79 80 81 82 83
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 27 28 29 30 31
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 19 20
[01:43:47] Can't kekulize mol.  Unkekulized ato

Explicit valence for atom # 1 B, 4, is greater than permitted
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
RDKit was unable to read the molecule.
Using the .

[01:43:47] Can't kekulize mol.  Unkekulized atoms: 5 10 14 15 18 19 24 28 49
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24
[01:43:47] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13


In [34]:
#########
#Obtain list from a file
#Input:
#(a) f_name file name
#Output:
#(a) list_from_file list from the file
#########
def obtain_list_from_file(f_name):
    list_from_file = []
    file_for_list = open(f_name, "r")
    for entry_add in file_for_list:
        list_from_file.append(entry_add.rstrip())
    file_for_list.close()
    return list_from_file

In [35]:
tr_list = obtain_list_from_file("/Users/dsharon/Documents/MIT/6.8701/Project/Data/For_ICLR_2024_02_05/PDB_Lists/train_pdb_ids.txt")
print(tr_list)
print(len(tr_list))

['4kz8', '1ajp', '3tu7', '2qi1', '4mvx', '3qxv', '4e35', '4dst', '3i5z', '4xxs', '3zot', '5fxq', '4oew', '3zt2', '3shz', '1gmy', '3rjc', '3rux', '3nhi', '5db0', '1m1b', '2x6i', '3bea', '4bi2', '4l32', '1g37', '2zjf', '4tya', '3tkh', '4qiz', '3wk8', '1ydk', '6g9n', '2q6f', '5ttu', '5a4c', '4baq', '1p57', '3rm9', '5a8z', '2r9m', '3qg6', '2x5o', '4ngp', '4anx', '5w88', '1ilh', '4yhm', '3pce', '3kme', '4wv6', '3p9l', '4uja', '3wff', '4z22', '2fai', '3f34', '4o42', '4xty', '1b5h', '4rx9', '3hub', '2xde', '2x8d', '1hsg', '4z0k', '4um3', '5kdf', '3ns9', '4wpf', '3kiv', '3t0x', '1x7q', '2z7g', '1wtg', '5hbs', '3nuj', '3ibl', '3sou', '1y2d', '5bry', '1g54', '2xj2', '2ay4', '5ekn', '5kkt', '3el7', '4uff', '1tt1', '2wi6', '5a2k', '4zy1', '4hge', '1y2h', '5igl', '5yid', '2cgx', '4h3c', '4c6v', '3uvm', '4u5j', '10gs', '3oyn', '1d3d', '4qq4', '2qu2', '1ghw', '5hed', '2fx8', '5g2n', '1nw4', '5mxr', '4oyo', '7lpr', '5ho7', '2l98', '5c85', '2chw', '5ak4', '3u9n', '4a7c', '2xda', '1rxp', '5ovc', '4ymx',

In [36]:
test_list = obtain_list_from_file("/Users/dsharon/Documents/MIT/6.8701/Project/Data/For_ICLR_2024_02_05/PDB_Lists/test_pdb_ids.txt")
print(test_list)
print(len(test_list))

['6o3x', '6oiq', '6qr2', '6pya', '6qrg', '6e4c', '6i63', '6r7d', '6dql', '6jmf', '6oxu', '6qlr', '6qmr', '6quv', '6qw8', '6gbw', '6qfe', '6jse', '5zr3', '6rnu', '6qqv', '6rpg', '6oir', '6st3', '6g5u', '6i76', '6oio', '6ggf', '6nrj', '6o9c', '6qr7', '6mo2', '6p8y', '6qr3', '6e5s', '6pz4', '6cf7', '6i65', '6pgp', '6cjp', '6oxw', '6qto', '6gge', '5zk7', '6i78', '6nw3', '6s55', '6jsn', '6g2c', '6e3z', '6cyg', '6ueg', '6ufo', '6e3n', '6nrf', '6nt2', '6jsg', '6pgo', '6ufn', '6s56', '6g2f', '6g2e', '6hor', '6os6', '6npi', '6bqd', '6gj5', '6i75', '6os5', '6oxy', '6uhu', '6d08', '6n8x', '6qlp', '6e6v', '6k3l', '6e3p', '6dz0', '6g2b', '6qlo', '6e3m', '6g27', '6gzy', '6g24', '6r4k', '6e13', '6gj7', '6pno', '6o0h', '6oxp', '6qqq', '6uim', '6m7h', '6e3o', '6qtq', '6iby', '6i64', '6qr0', '6mhd', '6h14', '6pka', '6py0', '6qrc', '6pnm', '6qr1', '6oip', '6qr9', '6t6a', '6o3y', '6jsf', '6c85', '6ggb', '6nsv', '6nrg', '6i74', '6n4b', '6nd3', '6i8t', '6jt3', '6ibx', '6hzd', '6s9x', '6e7m', '6g3c', '6jsz',